In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the FER dataset
data = pd.read_csv('./fer2013.csv')

# Extract the pixels and labels from the dataset
pixels = data['pixels'].tolist()
X = np.array([np.fromstring(pixel, dtype='int', sep=' ') for pixel in pixels])
X = X.reshape(-1, 48, 48, 1)
y = pd.get_dummies(data['emotion']).values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the ResNet model
def residual_block(x, filters, downsample=False):
    strides = (1, 1)
    if downsample:
        strides = (2, 2)
    
    y = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.ReLU()(y)
    
    y = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(y)
    y = layers.BatchNormalization()(y)
    
    if downsample:
        x = layers.Conv2D(filters, kernel_size=(1, 1), strides=(2, 2), padding='same')(x)
        
    out = layers.Add()([x, y])
    out = layers.ReLU()(out)
    return out

input_shape = X_train.shape[1:]
inputs = layers.Input(shape=input_shape)

# Initial Convolutional layer
x = layers.Conv2D(64, kernel_size=(3, 3), padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

# Residual blocks
x = residual_block(x, filters=64, downsample=False)
x = residual_block(x, filters=64, downsample=False)
x = residual_block(x, filters=64, downsample=False)

x = residual_block(x, filters=128, downsample=True)
x = residual_block(x, filters=128, downsample=False)
x = residual_block(x, filters=128, downsample=False)

x = residual_block(x, filters=256, downsample=True)
x = residual_block(x, filters=256, downsample=False)
x = residual_block(x, filters=256, downsample=False)

x = residual_block(x, filters=512, downsample=True)
x = residual_block(x, filters=512, downsample=False)
x = residual_block(x, filters=512, downsample=False)

# Classification head
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(7, activation='softmax')(x)

# Create the model
model = models.Model(inputs, outputs)

# Compile and train the model
model.compile(optimizer=optimizers.Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=64, epochs=20, validation_data=(X_test, y_test))

# Print the results
print("Training Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])

# Plot the training and validation accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt